In [ ]:
#### Reference: https://examples.dask.org/machine-learning/hyperparam-opt.html

In [ ]:
import pandas as pd
from dask import dataframe as dd 
from sklearn.linear_model import SGDClassifier
from dask_ml.model_selection import SuccessiveHalvingSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
import numpy as np
from dask.distributed import Client
from sklearn.datasets import make_circles
import dask.array as da
from dask_ml.model_selection import HyperbandSearchCV
from sklearn.neural_network import MLPClassifier
from dask_ml.model_selection import InverseDecaySearchCV

In [ ]:
from sklearn.datasets import make_circles
import numpy as np
import pandas as pd

X, y = make_circles(n_samples=30_000, random_state=0, noise=0.09)

pd.DataFrame({0: X[:, 0], 1: X[:, 1], "class": y}).sample(4_000).plot.scatter(
    x=0, y=1, alpha=0.2, c="class", cmap="bwr"
);

Create local dask cluster

In [ ]:
from distributed import Client
client = Client(processes=False, threads_per_worker=1,
                n_workers=4, memory_limit='2GB')
client

In [ ]:
from sklearn.utils import check_random_state

rng = check_random_state(42)
random_feats = rng.uniform(-1, 1, size=(X.shape[0], 4))
X = np.hstack((X, random_feats))
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=5_000, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier

In [ ]:
model = MLPClassifier()
params = {
    "hidden_layer_sizes": [
        (24, ),
        (12, 12),
        (6, 6, 6, 6),
        (4, 4, 4, 4, 4, 4),
        (12, 6, 3, 3),
    ],
    "activation": ["relu", "logistic", "tanh"],
    "alpha": np.logspace(-6, -3, num=1000),  # cnts
    "batch_size": [16, 32, 64, 128, 256, 512],
}

In [ ]:
n_examples = 4 * len(X_train)
n_params = 8
max_iter = n_params  # number of times partial_fit will be called
chunks = n_examples // n_params  # number of examples each call sees

max_iter, chunks

In [ ]:
import dask.array as da
X_train2 = da.from_array(X_train, chunks=chunks)
y_train2 = da.from_array(y_train, chunks=chunks)
X_train2

#### HyperbandSearchCV

In [ ]:
%%time
search = HyperbandSearchCV(model,params,max_iter=max_iter,patience=True,)
search.fit(X_train2, y_train2, classes=[0, 1, 2, 3])

In [ ]:
%%time
search.fit(X_train2, y_train2, classes=[0, 1, 2, 3])

In [ ]:
search.best_score_

#### SuccessiveHalvingSearchCV

In [ ]:
%%time
n, r = 10, 5 
search = SuccessiveHalvingSearchCV(model,params,n,r,max_iter=max_iter)
search.fit(X_train2, y_train2, classes=[0, 1, 2, 3])

In [ ]:
search.best_score_

#### InverseDecaysearchCV

In [ ]:
%%time
search = InverseDecaySearchCV(model, params,max_iter=max_iter)
search.fit(X_train2, y_train2, classes=[0, 1, 2, 3])

In [ ]:
search.best_score_